# AI-Platformを利用したTitanicデータ予測

GCP の AI-Platform へモデルをデプロイし予測をします。
※ 2020/02/12 時点での情報及び動作確認となります。

## GCP アカウント登録

[【画像で説明】Google Cloud Platform (GCP)の無料トライアルでアカウント登録](https://qiita.com/komiya_____/items/14bd06d0866f182ae912)

<br>


<br>

## Google Cloud SDK のインストール

[Google Cloud SDK のインストール ~ 初期化](https://qiita.com/komiya_____/items/5af0dcc8639fad9fee29)

<br>

## ログイン

gcloud コマンドをつかって GCP をいじるための認証になります。

In [0]:
! gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?code_challenge=Eftqw60jPIBCEwsg8jecxOuDp1zNN_kbmE6c-KtxbeE&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


Enter verification code: 4/wgEezvqzjrdkmCGUYleODpqn6kVrOBT3Iid_NONAYX9Uj3WWLo0hJys
ERROR: There was a problem with web authentication.
ERROR: (gcloud.auth.login) invalid_grantInvalid code verifier.


<br>

# プロジェクト



## プロジェクト ID とプロジェクトネームを設定

ID はカブり禁止です。

In [0]:
PROJECT_ID = 'watashino-pjid-a' #@param {type: "string"}
PROJECT_NAME = 'watashino-pjname-a' #@param {type: "string"}

<br>

# 作成

In [0]:
! gcloud projects create $PROJECT_ID \
--name $PROJECT_NAME

Create in progress for [https://cloudresourcemanager.googleapis.com/v1/projects/watashi-no-project-id-a].
Enabling service [cloudapis.googleapis.com] on project [watashi-no-project-id-a]...
Operation "operations/acf.c17e02e4-6ad3-44e3-a8d5-dfa3adc990d1" finished successfully.


<br>

## プロジェクトをアクティブに
操作対象のプロジェクトに。

In [0]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


<br>

## 確認
- project = watashi-no-project-id

となっていればOK。

In [0]:
! gcloud config list

[component_manager]
disable_update_check = True
[compute]
gce_metadata_read_timeout_sec = 0
[core]
project = watashino-pjid-a

Your active configuration is: [default]


<br>

# リージョン、ゾーン、インタープリターを設定

AI Platform のオンライン予測が使えるリージョンは以下。
  - us-central1
  - us-east1
  - us-east4
  - asia-northeast1
  - europe-west1

インタープリターはローカルトレーニング　 / 予測の際にpython３系を使うように指定。


In [0]:
REGION = 'us-central1' #@param {type: 'string'}
ZONE = 'us-central1-a' #@param {type: 'string'}

In [0]:
! gcloud config set compute/region $REGION
! gcloud config set compute/zone $ZONE
! gcloud config set ml_engine/local_python $(which python3)

Updated property [compute/region].
Updated property [compute/zone].
Updated property [ml_engine/local_python].


<br>

## 確認
  - region = us-central1
  - zone = us-central1-a
  - local_python = /usr/bin/python3

となっていればOK。

In [0]:
! gcloud config list

[component_manager]
disable_update_check = True
[compute]
gce_metadata_read_timeout_sec = 0
region = us-central1
zone = us-central1-a
[core]
account = edsa9654@gmail.com
project = watashi-no-project-id-a
[ml_engine]
local_python = /usr/bin/python3

Your active configuration is: [default]


<br>

# トレーニング用のコード一式をクローン

In [0]:
! git clone https://github.com/komiyakomiyakomiya/titanic_prediction_on_gcp.git

Cloning into 'titanic_prediction_on_gcp'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 143 (delta 68), reused 117 (delta 42), pack-reused 0
Receiving objects: 100% (143/143), 147.92 KiB | 1.30 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [0]:
# ! rm -r titanic_prediction_on_gcp/

<br>

# modelを保存するディレクトリを作成

In [0]:
import os

os.makedirs('./titanic_prediction_on_gcp/working/models/', exist_ok=True)

<br>

# ローカルでトレーニング
トレーニング後のモデルが `./titanic_prediction_on_gcp/working/models/model.pkl` として保存されます。

In [0]:
! gcloud ai-platform local train \
--package-path titanic_prediction_on_gcp/working/ \
--module-name working.predict_xgb 

/content/titanic_prediction_on_gcp/working
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
[0]	train-logloss:0.557009	eval-logloss:0.616411
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 10 rounds.
[1]	train-logloss:0.474548	eval-logloss:0.571008
[2]	train-logloss:0.422645	eval-logloss:0.550512
[3]	train-logloss:0.38807	eval-logloss:0.538894
[4]	train-logloss:0.364927	eval-logloss:0.542051
[5]	train-logloss:0.343597	eval-logloss:0.549628
[6]	train-logloss:0.332187	eval-logloss:0.560026
[7]	train-logloss:0.320152	eval-logloss:0.570064
[8]	train-logloss:0.309393	eval-logloss:0.584526
[9]	train-logloss:0.303566	eval-logloss:0.593687
[10]	train-logloss:0.296048	eval-logloss:0.601536
[11]	train-logloss:0.289636	eval-logloss:0.60886
[12]	train-logloss:0.28482	e

<br>

# モデルをデプロイ

# 請求先アカウントの設定
バケットを作成するために必要になります。

ここから設定

<img width="995" src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/287998/d7f69423-b671-6816-e185-8b7ede2f7b5e.png">

以下のポップアップが出なければ、請求先アカウント設定済みなのでスキップしてOKです。

<img width="876" src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/287998/14f33fa4-7da2-2b02-206e-5feb5ece3944.png">

<img width="878" src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/287998/72de2c43-4306-0b51-819b-66f407979281.png">

<br>

## バケットの作成
保存したモデルをアップロードするため、GCSにバケットを作ります。
※ 請求先アカウントを設定していないと403エラーになります。

In [0]:
BUCKET_NAME = 'watashi-no-bucket-name-a' #@param {type: 'string'}

In [0]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Creating gs://watashi-no-bucket-name-a/...


In [0]:
! gsutil ls -la

gs://watashi-no-bucket-name/


In [0]:
! gsutil cp './titanic_prediction_on_gcp/working/models/model.pkl' gs://$BUCKET_NAME/models/model.pkl

Copying file://./titanic_prediction_on_gcp/working/models/model.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 12.8 KiB/ 12.8 KiB]                                                
Operation completed over 1 objects/12.8 KiB.                                     


In [0]:
! gsutil ls gs://$BUCKET_NAME/models/

gs://watashi-no-bucket-name-a/models/model.pkl


## APIを有効化
AI-Platform APIを使うための設定。
以下の２つを有効化。

  - ml.googleapis.com
  - compute.googleapis.com



In [0]:
! gcloud services enable ml.googleapis.com
! gcloud services enable compute.googleapis.com

<br>

## 確認
  - compute.googleapis.com            Compute Engine API
  - ml.googleapis.com                 AI Platform Training & Prediction API

があればOK。

In [0]:
! gcloud services list --enabled

NAME                              TITLE
bigquery.googleapis.com           BigQuery API
bigquerystorage.googleapis.com    BigQuery Storage API
cloudapis.googleapis.com          Google Cloud APIs
clouddebugger.googleapis.com      Stackdriver Debugger API
cloudtrace.googleapis.com         Stackdriver Trace API
compute.googleapis.com            Compute Engine API
datastore.googleapis.com          Cloud Datastore API
logging.googleapis.com            Stackdriver Logging API
ml.googleapis.com                 AI Platform Training & Prediction API
monitoring.googleapis.com         Stackdriver Monitoring API
oslogin.googleapis.com            Cloud OS Login API
servicemanagement.googleapis.com  Service Management API
serviceusage.googleapis.com       Service Usage API
sql-component.googleapis.com      Cloud SQL
storage-api.googleapis.com        Google Cloud Storage JSON API
storage-component.googleapis.com  Cloud Storage


# サービスアカウント

In [0]:
SERVICE_ACCOUNT_NAME = 'watashino-service-account-name' #@param {type: 'string'}
SEFVICE_ACCOUNT_DISPLAY_NAME = 'watashino--service-account-display-name' #@param {type: 'string'}

! gcloud iam service-accounts create $SERVICE_ACCOUNT_NAME \
--display-name $SEFVICE_ACCOUNT_DISPLAY_NAME

Created service account [watashino-service-account-name].


In [0]:
! gcloud projects get-iam-policy watashi-no-project-a

ERROR: (gcloud.projects.get-iam-policy) User [edsa9654@gmail.com] does not have permission to access project [watashi-no-project-a:getIamPolicy] (or it may not exist): The caller does not have permission


<br>

## モデルリソース / バージョンリソースの作成
モデルリソースとバージョンリソースを作成し、アップロードしたmodel.pklと紐付けます。

In [0]:
MODEL_NAME = 'model_xgb' #@param {type: 'string'}
MODEL_VERSION = 'v1' #@param {type: 'string'}

In [0]:
! gcloud ai-platform models create $MODEL_NAME \
--regions $REGION

Created ml engine model [projects/watashi-no-project-id-a/models/model_xgb].


In [0]:
! gcloud ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --origin gs://$BUCKET_NAME/models/ \
  --runtime-version 1.14 \
  --framework xgboost \
  --python-version 3.5

ERROR: (gcloud.ai-platform.versions.create) argument --model: expected one argument
Usage: gcloud ai-platform versions create VERSION --model=MODEL [optional flags]
  optional flags may be  --async | --config | --description | --framework |
                         --help | --labels | --origin | --python-version |
                         --runtime-version | --staging-bucket

For detailed information on this command and its flags, run:
  gcloud ai-platform versions create --help


In [0]:
from IPython.display import Image
from IPython.display import display_png


print('\nあなたはタイタニックに乗ったら...')
display_png(Image('/content/titanic_prediction_on_gcp/images/dead.png'))


あなたはタイタニックに乗ったら...
